# 0. Imports

In [23]:
import sys
sys.path.append('../APP/back-end/')

import pandas as pd
from player_profiling.preprocessor import load_data_fifa23, filter_data, preprocess
from sklearn.cluster import DBSCAN, HDBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import SpectralClustering


# 1. Loading data

In [2]:
data = load_data_fifa23()


/Users/luizfelipedossantos/code/micheledeantoni/APP/back-end/player_profiling/preprocessor.py:14: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../raw_data/male_players_23.csv',


,player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url
0,158023,/player/158023/lionel-messi/230009,23,9,2023-01-13,L. Messi,Lionel Andrés Messi Cuccittini,RW,91,91,...,63+3,63+3,64+3,59+3,50+3,50+3,50+3,59+3,19+3,https://cdn.sofifa.net/players/158/023/23_120.png
1,165153,/player/165153/karim-benzema/230009,23,9,2023-01-13,K. Benzema,Karim Benzema,"CF, ST",91,91,...,64+3,64+3,64+3,60+3,55+3,55+3,55+3,60+3,18+3,https://cdn.sofifa.net/players/165/153/23_120.png
2,188545,/player/188545/robert-lewandowski/230009,23,9,2023-01-13,R. Lewandowski,Robert Lewandowski,ST,91,91,...,66+3,66+3,64+3,61+3,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/23_120.png
3,192985,/player/192985/kevin-de-bruyne/230009,23,9,2023-01-13,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,...,79+3,79+3,78+3,74+3,68+3,68+3,68+3,74+3,21+3,https://cdn.sofifa.net/players/192/985/23_120.png
4,231747,/player/231747/kylian-mbappe/230009,23,9,2023-01-13,K. Mbappé,Kylian Mbappé Lottin,"ST, LW",91,95,...,63+3,63+3,67+3,63+3,54+3,54+3,54+3,63+3,18+3,https://cdn.sofifa.net/players/231/747/23_120.png


In [11]:
data_fifa23 = data.copy()
data_fifa23.head()


,player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url
0,158023,/player/158023/lionel-messi/230009,23,9,2023-01-13,L. Messi,Lionel Andrés Messi Cuccittini,RW,91,91,...,63+3,63+3,64+3,59+3,50+3,50+3,50+3,59+3,19+3,https://cdn.sofifa.net/players/158/023/23_120.png
1,165153,/player/165153/karim-benzema/230009,23,9,2023-01-13,K. Benzema,Karim Benzema,"CF, ST",91,91,...,64+3,64+3,64+3,60+3,55+3,55+3,55+3,60+3,18+3,https://cdn.sofifa.net/players/165/153/23_120.png
2,188545,/player/188545/robert-lewandowski/230009,23,9,2023-01-13,R. Lewandowski,Robert Lewandowski,ST,91,91,...,66+3,66+3,64+3,61+3,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/23_120.png
3,192985,/player/192985/kevin-de-bruyne/230009,23,9,2023-01-13,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,...,79+3,79+3,78+3,74+3,68+3,68+3,68+3,74+3,21+3,https://cdn.sofifa.net/players/192/985/23_120.png
4,231747,/player/231747/kylian-mbappe/230009,23,9,2023-01-13,K. Mbappé,Kylian Mbappé Lottin,"ST, LW",91,95,...,63+3,63+3,67+3,63+3,54+3,54+3,54+3,63+3,18+3,https://cdn.sofifa.net/players/231/747/23_120.png


# 1.5 Functions

## Create Label Column

In [24]:
def create_label_column(labels):
    data_fifa23['label'] = labels


## Show Groups

In [26]:
def show_label_groups():
    for group in sorted(data_fifa23.label.unique()):

        print("================")
        print(f"players belonging to group {group}")
        print("================")
        print(data_fifa23[data_fifa23['label'] == group].sample(5)[['short_name']])


# 2. Preprocessing

## Filtering columns

In [12]:
data_fifa23_filtered = filter_data(data_fifa23)


/Users/luizfelipedossantos/code/micheledeantoni/APP/back-end/player_profiling/preprocessor.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  characteristics['goalkeeping_speed'].fillna(0, inplace=True)


## Preprocessing Data

In [13]:
data_fifa23_preproc = preprocess(data_fifa23_filtered)


In [14]:
data_fifa23_preproc.head()


,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.258808,2.820995,3.518769,...,-0.941176,-0.540541,-0.763158,0.0,0.045455,0.102273,0.152174,0.136364,0.078652,0.0
1,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.160309,2.743755,2.767791,...,-0.264706,-0.837838,-0.921053,0.0,0.125000,0.102273,0.043478,0.034091,0.067416,0.0
2,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.667814,2.975476,2.338661,...,-0.500000,-0.351351,-0.894737,0.0,0.147727,0.045455,0.119565,0.068182,0.101124,0.0
3,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.569314,2.743755,3.840616,...,0.382353,0.270270,0.000000,0.0,0.147727,0.125000,0.043478,0.090909,0.134831,0.0
4,1.0,0.0,0.0,1.0,1.0,0.0,0.0,2.834795,2.820995,2.445943,...,-0.764706,-0.567568,-0.552632,0.0,0.125000,0.034091,0.065217,0.102273,0.056180,0.0


# 3. Fitting model DBScan

In [10]:
dbscan = DBSCAN(eps=5, min_samples=10).fit(data_fifa23_preproc)

labels = dbscan.labels_


In [11]:
labels


array([-1, -1, -1, ...,  1,  1,  1])

## Creating label column on the data

In [12]:
create_label_column(labels)

data_fifa23.head()


,player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,...,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,labels
0,158023,/player/158023/lionel-messi/230009,23,9,2023-01-13,L. Messi,Lionel Andrés Messi Cuccittini,RW,91,91,...,63+3,64+3,59+3,50+3,50+3,50+3,59+3,19+3,https://cdn.sofifa.net/players/158/023/23_120.png,-1
1,165153,/player/165153/karim-benzema/230009,23,9,2023-01-13,K. Benzema,Karim Benzema,"CF, ST",91,91,...,64+3,64+3,60+3,55+3,55+3,55+3,60+3,18+3,https://cdn.sofifa.net/players/165/153/23_120.png,-1
2,188545,/player/188545/robert-lewandowski/230009,23,9,2023-01-13,R. Lewandowski,Robert Lewandowski,ST,91,91,...,66+3,64+3,61+3,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/23_120.png,-1
3,192985,/player/192985/kevin-de-bruyne/230009,23,9,2023-01-13,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,...,79+3,78+3,74+3,68+3,68+3,68+3,74+3,21+3,https://cdn.sofifa.net/players/192/985/23_120.png,-1
4,231747,/player/231747/kylian-mbappe/230009,23,9,2023-01-13,K. Mbappé,Kylian Mbappé Lottin,"ST, LW",91,95,...,63+3,67+3,63+3,54+3,54+3,54+3,63+3,18+3,https://cdn.sofifa.net/players/231/747/23_120.png,-1


In [13]:
data_fifa23.label.value_counts()


 1    18234
 0     2296
-1      116
 2        5
Name: labels, dtype: int64

## Seeing what the groups represent

In [14]:
show_label_groups()


players belonging to group -1
      short_name
53      K. Coman
160    H. Ziyech
87      P. Foden
113  J. Grealish
186    J. Sancho
players belonging to group 0
            short_name
161122  Fidel Castillo
5690         L. Zidane
17253            L. Øy
1847        M. Andújar
16671     J. Ojrzyński
players belonging to group 1
          short_name
5427    A. Sharahili
1808      N. Seiwald
6894    Miguel Baeza
151258        Míchel
14330         Pastor
players belonging to group 2
        short_name
142        L. Díaz
194       C. Gakpo
188       Raphinha
268  Ferran Torres
273       D. Núñez


# 4. Fitting model HDBScan

In [26]:
hdb = HDBSCAN(min_cluster_size=10).fit(data_fifa23_preproc)

labels = hdb.labels_

labels


array([ 0, -1, -1, ...,  0,  0,  0])

## Creating label column on the data

In [27]:
create_label_column(labels)


In [28]:
data_fifa23.label.value_counts()


 0    18329
 1     2303
-1       19
Name: labels, dtype: int64

## Seeing what the groups represent

In [29]:
show_label_groups()


players belonging to group -1
        short_name
10        J. Oblak
20        T. Kroos
2   R. Lewandowski
6         M. Salah
16         Alisson
players belonging to group 0
         short_name
5653     P. Pomykal
6268        N. Sarr
10753   Y. González
5405       S. Simič
106045      C. Coxe
players belonging to group 1
        short_name
14616     J. Pukaj
9739      D. Sappa
1426   Joel Robles
9735      M. Peano
1444       T. Horn


# 5. Fitting model Spectral Clustering

In [27]:
clustering = SpectralClustering(n_clusters=30,
                                assign_labels='kmeans',
                                affinity='nearest_neighbors',
                                n_init=5,
                                n_jobs=-1).fit(data_fifa23_preproc)

labels = clustering.labels_

labels


/Users/luizfelipedossantos/.pyenv/versions/3.10.6/envs/APP/lib/python3.10/site-packages/sklearn/manifold/_spectral_embedding.py:273: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


array([17, 17, 17, ..., 12, 12, 16], dtype=int32)

## Creating label column on the data

In [28]:
create_label_column(labels)


In [29]:
data_fifa23.label.value_counts()


label
1     2146
28    2079
9     1662
14    1433
4     1408
5     1402
6     1222
0     1205
2      873
8      764
22     673
20     637
29     617
3      599
27     433
25     431
12     358
24     298
15     257
21     249
11     233
23     227
7      226
16     222
13     201
19     196
26     194
10     184
17     150
18      72
Name: count, dtype: int64

## Seeing what the groups represent

In [30]:
show_label_groups()


players belonging to group 0
           short_name
32809  M. Al Shammari
10400       I. Lomami
14884      O. Dodgson
28443         C. Áñez
6939    H. Matriciani
players belonging to group 1
         short_name
11655    G. Fuengos
15511   J. González
12186     J. Bowden
11555     A. Iovine
104540  Víctor Olmo
players belonging to group 2
          short_name
2466    C. Pellerano
862       C. Kouyaté
22465       C. Fuchs
1798   Nico González
756      J. Andersen
players belonging to group 3
               short_name
17474          H. Melland
17809  E. Lalchhanchhuaha
17674  F. Loftesnes-Bjune
17389          I. Parente
16675      I. Lewandowski
players belonging to group 4
          short_name
11070        S. Guri
15348   S. Măzărache
12097       M. Kolar
13021    D. Nlundulu
10183  B. Hollerbach
players belonging to group 5
          short_name
5467    F. Dagerstål
6723     G. Guiffrey
10662     A. Obileye
6229        A. Grant
152972    E. Kwateng
players belonging to group 6
         sh